# Classificador

## 1. Importar dados

Para criar um classificador formatar os dados da sequinte maneira:

moleculas -> uma lista/array com todos os smiles do seu banco de dados

classes -> uma lista/array de zeros e uns representando antibiótico e não antibiótico, respectivamente

O classificador vai identificar traços das moleculas e associá-los às classes, então cada molécula deverá ter o mesmo índice que sua classe. 

bibliotecas sugeridas:
 - pandas
 - numpy

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk

#Importar dados
df = pd.read_csv('/home/joao/documentos/IC-Joao/classificador/data_teste.csv',sep=';', encoding='latin-1')

#Definir a coluna das classes
labels = np.asarray(df.Classe)

#Selecionar SMILES sem as classes
df_selected = df.drop(["Classe"], axis=1)

## 2. Processamento dos dados

### 2.1 Encoding/padding

Nessa etapa acontece a codificação das moleculas. Pode ser tokens, one hot vector, integer vector. Você decide.

É importante que ao final do processamento todos os tokens tenham o mesmo comprimento, então um processo chamado padding precisa ser aplicado.

Por convenção as variáveis em machine learning assumem esses nomes:

X -> uma lista/array com todos os smiles codificados

y -> uma lista/array de zeros e uns representando antibiótico e não antibiótico, respectivamente


Bibliotecas sugeridas:
 - numpy
 - sklearn
 

In [2]:
#Encoding dos labels
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le.fit(labels)

labels = le.transform(labels)


#Descobrir maior smile da base de dados
x = 0
longest_smile = '0'

while x<len(df_selected.Smiles):
    if len(df_selected.Smiles[x])>len(longest_smile):
        longest_smile = df_selected.Smiles[x]
    else:
        longest_smile = longest_smile
    x = x+1

#Fazer padding

code_padding = 'A'
lista_padding = []
y = 0

while y<len(df_selected.Smiles):
    if len(df_selected.Smiles[y])<len(longest_smile):
        lista_padding.append(df_selected.Smiles[y].rjust(len(longest_smile),code_padding))
    y = y+1

#Criar vocabulário para tokenização
atoms = [
         'H','B', 'C', 'N', 'O', 'P', 'S', 'F', 'Cl', 'Br', 'I','Na','Li','Ca','Al','Ag','Se','Mg','K','Zn'
        ]

special = [
        '(', ')', '[', ']', '=', '#', '@', '*', '%', '0', '1', '2',
        '3', '4', '5', '6', '7', '8', '9', '.', '/', '\\', '+', '-',
         'c', 'n', 'o', 's','p'
        ]

padding = ['G', 'A', 'E'] #Go, Padding ,End
table = sorted(atoms, key=len, reverse=True) + special + padding
table_len = len(table)

#Funções de tokenização e criação de one-hot encode
def tokenize(smiles):
    N = len(smiles)
    i = 0
    j= 0
    token = []
    while (i < N):
        for j in range(table_len):
            symbol = table[j]
            if symbol == smiles[i:i + len(symbol)]:
                token.append(symbol)
                i += len(symbol)
                break
    return token

def one_hot_encode(trans_char): #create one hot encode table
    transl_one_hot = {} #create dictionary
    for i, char in enumerate(table): #create one hot encode vector for each character
        lista = np.zeros(table_len) #create zero list for each char
        lista[i] = 1 #set 1 on the correct position
        transl_one_hot[char] = lista #save list in dictionary

    result = np.array([transl_one_hot[s] for s in trans_char]) #find the vector corresponding to the character
    result = result.reshape(1, result.shape[0], result.shape[1])
    #print("\nTransl_one_hot:\n",transl_one_hot,"\n")
    return result

    #Aplicação da função para obter os vetores de cada smiles
smiles = []
z = 0

while z<len(df_selected.Smiles):
    smiles.append(one_hot_encode(tokenize(df_selected.Smiles[z])))
    z = z+1


## 3. Holdout: train/test split

Depois do processamento, seu banco de dados precisa ser dividio em traino e teste. O _Holdout_ é o ponto de separação. Geralmente os dados são dividos em dados em 70/30. 70% para treino e 30% para teste.

Agora as variáveis serão separadas pelos conjuntos criados:

X_treino -> 70% dos smiles codificados

X_teste  -> 30% dos smiles codificados

y_treino -> 70% das classes

y_teste  -> 30% das classes

Bibliotecas sugeridas:
 - numpy
 - sklearn

In [ ]:
from sklearn.model_selection import train_test_split

smiles_train, smiles_test, labels_train, labels_test = train_test_split(
    smiles, labels, 
    test_size=0.30, random_state=42)

## 4. Modelo

Depois de preparar os dados, precisamos criar um variável que contem nosso modelo. Estude o tensoflow/keras, mais especificamente a função Sequential(), para criação dos modelos.

Arquiteturas sugeridas:
- MultiLayer Perceptron (MLP)
- Convulional Neural Networks (CNN)

Não esqueça de adicionar as métricas precision, recall, accuracy e loss. Para essa ultima, você precisa estudar sobre as funções de perda usadas para classificação binária (0,1).

Depois de construído use o método .fit() para treinar seu classificador

In [ ]:
#Código aqui

## 5 Avaliação

Apos o treinamento, estude como fazer a predição no conjunto de teste com o modelo pronto. Com esses dados em mãos você pode construir uma matriz de confusão que mostra o desempenho do seu modelo

In [ ]:
#Código aqui